In [43]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch import flatten
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.autograd import Variable


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [96]:
class CNN(nn.Module):
    def __init__(self, in_channels = 1, num_classes = 10):
        super(CNN, self).__init__()
        self.C1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), stride=1, padding='same')
        self.pool = nn.MaxPool2d(kernel_size=(3,3), stride=(2,2))
        self.C2 = nn.Conv2d(in_channels=8, out_channels=8, kernel_size=(3,3), stride=1, padding='same')
        self.C3 = nn.Conv2d(in_channels=8, out_channels=8, kernel_size=(3,3), stride=1, padding='same')
        self.C4 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=2)
        self.C5 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3,3), stride=1, padding='same')
        self.C6 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3,3), stride=1, padding='same')
        self.ave_pool = nn.AvgPool2d(2,1)
        self.fc1 = nn.Linear(400, 400)
        self.fc2 = nn.Linear(400, num_classes)

    def forward(self, x):
        x = nn.functional.relu(self.C1(x))
        x = self.pool(x)

        shortcut = x
        x = nn.functional.relu(self.C2(x))
        x = nn.functional.relu(self.C3(x) + shortcut)
        x = nn.functional.relu(self.C4(x))

        shortcut = x
        x = nn.functional.relu(self.C5(x))
        x = nn.functional.relu(self.C6(x))
        x = self.ave_pool(x + shortcut)
        #x = x.reshape(x.shape[0],-1)
        x = flatten(x,1)
        x = nn.functional.softmax(self.fc1(x))
        x = self.fc2(x)
        
        return x


In [98]:
learning_rate = 0.001
num_classes = 10
batch_size = 64
num_epochs = 100

In [97]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [99]:
transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,)),
                   ])

In [100]:
train_dataset = datasets.MNIST(root='./dataset/', train=True, transform=transform, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='./dataset/', train=False, transform=transform, download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)


In [101]:
trainSteps = len(train_loader.dataset) // batch_size
trainSteps

937

In [102]:
#### Train
for epoch in range(num_epochs):
    model.train()   
    
    totalTrainLoss = 0

    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)

        scores = model(data)
        loss = criterion(scores, targets)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        totalTrainLoss += loss.item()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {(totalTrainLoss/trainSteps):.6f}')

        

C:\Users\Admin\AppData\Local\Temp/ipykernel_7824/3646285791.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = nn.functional.softmax(self.fc1(x))


Epoch 1/100, Loss: 1.778180
Epoch 2/100, Loss: 1.158890
Epoch 3/100, Loss: 0.950110
Epoch 4/100, Loss: 0.867848
Epoch 5/100, Loss: 0.825231
Epoch 6/100, Loss: 0.802645
Epoch 7/100, Loss: 0.782166
Epoch 8/100, Loss: 0.714443
Epoch 9/100, Loss: 0.588646
Epoch 10/100, Loss: 0.466365
Epoch 11/100, Loss: 0.371137
Epoch 12/100, Loss: 0.299525
Epoch 13/100, Loss: 0.235554
Epoch 14/100, Loss: 0.185229
Epoch 15/100, Loss: 0.150679
Epoch 16/100, Loss: 0.125110
Epoch 17/100, Loss: 0.108113
Epoch 18/100, Loss: 0.092724
Epoch 19/100, Loss: 0.079567
Epoch 20/100, Loss: 0.073202
Epoch 21/100, Loss: 0.066961
Epoch 22/100, Loss: 0.059608
Epoch 23/100, Loss: 0.053578
Epoch 24/100, Loss: 0.051944
Epoch 25/100, Loss: 0.047224
Epoch 26/100, Loss: 0.044333
Epoch 27/100, Loss: 0.041251
Epoch 28/100, Loss: 0.038306
Epoch 29/100, Loss: 0.036537
Epoch 30/100, Loss: 0.034584
Epoch 31/100, Loss: 0.035067
Epoch 32/100, Loss: 0.031522
Epoch 33/100, Loss: 0.029710
Epoch 34/100, Loss: 0.032017
Epoch 35/100, Loss: 0.0

In [103]:
def accuracy(loader, model):
    if loader.dataset.train:
        print("---Trainning Accuracy---")
    else:
        print("---Test Accuracy---")
    
    num_correct = 0
    num_sample = 0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions==y).sum()
            num_sample  += predictions.size(0)
        print(f"Got {num_correct}/{num_sample} with accuracy {round(float(num_correct/num_sample),4)}")
    model.train()


In [104]:
accuracy(train_loader,model)
accuracy(test_loader, model)

---Trainning Accuracy---


C:\Users\Admin\AppData\Local\Temp/ipykernel_7824/3646285791.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = nn.functional.softmax(self.fc1(x))


Got 59666/60000 with accuracy 0.9944
---Test Accuracy---
Got 9840/10000 with accuracy 0.984


In [109]:
# predict
def predict_(test_loader, model):
    predict = []
    true_label = []
    n = np.random.randint(len(test_loader))
    i = 0
    model.eval()
    with torch.no_grad():
        for x, y in test_loader:
            x = x.to(device=device)
            true_label = y
            y = y.to(device=device)
            scores = model(x)
            _, predictions = scores.max(1)
        
            predict = predictions.to('cpu')
            i+=1
            if i == n:
                break
    model.train()
    return predict, true_label


In [112]:
#Predict 
predict, true_label = predict_(test_loader, model)
predict = [np.array(i,dtype=int) for i in predict]
true_label = [np.array(i,dtype=int) for i in true_label]
t = np.random.randint(len(predict))
print("---Random Predict---")
print('|  Number predict  |')
print('____________________')
print("|"," "*6 ,predict[t], " "* 7, "|")
print('____________________')
print('|True number label |')
print('____________________')
print("|"," "*6, true_label[t]," "*7, "|")
print('____________________')
print("        -------Wrong Predict-------")
print("____________________________________________")
print("| Predict Number    |  True Label's Number |")
print("____________________________________________")
for i in range(batch_size):
    if predict[i]!=true_label[i]:
        print("|       ",predict[i], "  " *4,"|"," "* 9, true_label[i], " "* 8,"|")
        print("____________________________________________")



C:\Users\Admin\AppData\Local\Temp/ipykernel_7824/3646285791.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = nn.functional.softmax(self.fc1(x))


---Random Predict---
|  Number predict  |
____________________
|        9         |
____________________
|True number label |
____________________
|        9         |
____________________
        -------Wrong Predict-------
____________________________________________
| Predict Number    |  True Label's Number |
____________________________________________
|        3          |           5          |
____________________________________________
|        2          |           3          |
____________________________________________
